In [1]:
from faker import Faker
import json
import sqlite3
import random
import secrets

fake = Faker()

In [2]:
# gacha function to generate employee role
def gacha_role():
    role = [
        "Manager",
        "Assistant Manager",
        "Supervisor",
        "Accountant",
        "Clerk",
        "Salesman",
        "Driver",
        "Storkeeper"
    ]
    
    weights = [
        0.05,  # Manager
        0.10,  # Assistant Manager
        0.15,  # Supervisor
        0.20,  # Accountant
        0.20,  # Clerk
        0.15,  # Salesman
        0.10,  # Driver
        0.05   # Storkeeper
    ]
    
    selected_role = random.choices(role, weights, k=1)[0]
    
    return selected_role

# gacha gender
def gacha_gender():
    gender = ['Male', 'Female']
    weights = [0.5, 0.5]
    
    return random.choices(gender, weights, k=1)[0]

In [3]:
# cyrpto id generator
def crypto_id():
    return secrets.token_hex(36)

# generate state
def gacha_state():
    state = [
        "Johor",
        "Kedah",
        "Kelantan",
        "Malacca",
        "Negeri Sembilan",
        "Pahang",
        "Penang",
        "Perak",
        "Perlis",
        "Sabah",
        "Sarawak",
        "Selangor",
        "Terengganu",
        "Kuala Lumpur",
        "Labuan",
        "Putrajaya"
    ]
    
    return random.choice(state)

# generate epf number
def gacha_epf():
    return random.randint(100000000, 999999999)

# generate socso number
def gacha_socso():
    return random.randint(100000000, 999999999)

# generate bank account number
def gacha_bank_account():
    return fake.iban()

# get marital status
def gacha_marital_status():
    marital_status = ['Single', 'Married', 'Divorced', 'Widowed']
    weights = [0.5, 0.3, 0.1, 0.1]
    
    return random.choices(marital_status, weights, k=1)[0]

# get date of birth
def gacha_dob():
    return fake.date_of_birth(minimum_age=18, maximum_age=65).strftime('%Y-%m-%d')

# use faker to get nationality
def gacha_nationality():
    return fake.country()

# resident status: citizen, permanent resident, foreigner
def gacha_resident_status():
    resident_status = ['Citizen', 'Permanent Resident', 'Foreigner']
    weights = [0.7, 0.25, 0.05]
    
    return random.choices(resident_status, weights, k=1)[0]

In [4]:
# fake employee data generator
def fake_employee():
    fake = Faker()
    
    fake_employee = {
    "id": crypto_id(),
    "age": fake.random_int(min=18, max=65),
    "name": fake.name(),
    "gender": gacha_gender(),
    "date_of_birth": gacha_dob(),
    "address": fake.address(),
    "state": gacha_state(),
    "phone_number": fake.phone_number(),
    "email": fake.email(),
    "nationality": gacha_nationality(),
    "resident_status": gacha_resident_status(),
    "marital_status": gacha_marital_status(),
    "role": gacha_role(),
    "salary": fake.random_int(min=1000, max=10000),
    "hire_date": fake.date_this_decade().strftime("%Y-%m-%d"),
    "epf_number": gacha_epf(),
    "socso_number": gacha_socso(),
    "bank_account": gacha_bank_account()
    }
    
    return fake_employee

# generate multiple fake employee JSON
def generate_multiple_fake_employee(n):
    employees = [fake_employee() for _ in range(n)]
    return json.dumps(employees, indent=4)

In [5]:
print(generate_multiple_fake_employee(1))

[
    {
        "id": "20b5e225be5fa10545e9466eb9ecb97354a0f465ab9100be55e8c86a064e9230b3b0e9e3",
        "age": 59,
        "name": "Sarah Mercado",
        "gender": "Female",
        "date_of_birth": "1993-08-16",
        "address": "65605 Johnson Rapid\nNew Sherri, OH 78973",
        "state": "Putrajaya",
        "phone_number": "832.370.6099",
        "email": "vickie30@example.net",
        "nationality": "Lebanon",
        "resident_status": "Permanent Resident",
        "marital_status": "Single",
        "role": "Driver",
        "salary": 3097,
        "hire_date": "2020-02-26",
        "epf_number": 869084057,
        "socso_number": 594683025,
        "bank_account": "GB71DNKL46864476735354"
    }
]


In [6]:
# insert fake employee data to sqlite3
conn = sqlite3.connect('database/database.db')
c = conn.cursor()

c.execute(
    '''
    CREATE TABLE IF NOT EXISTS employees (
        id TEXT PRIMARY KEY NOT NULL,
        age INTEGER NOT NULL,
        name TEXT NOT NULL,
        gender TEXT NOT NULL,
        date_of_birth DATE NOT NULL,
        address TEXT NOT NULL,
        state TEXT NOT NULL,
        phone_number TEXT NOT NULL,
        email TEXT NOT NULL,
        nationality TEXT NOT NULL,
        resident_status TEXT NOT NULL,
        marital_status TEXT NOT NULL,
        role TEXT NOT NULL,
        salary INTEGER NOT NULL,
        hire_date DATE NOT NULL,
        epf_number INTEGER NOT NULL,
        socso_number INTEGER NOT NULL,
        bank_account TEXT NOT NULL
    )
    '''
)

json_data = generate_multiple_fake_employee(200)

# save the json data as json
with open('json_data/employees.json', 'w') as f:
    f.write(json_data)

for employee in json.loads(json_data):
    c.execute(
        '''INSERT INTO employees(id, age, name, gender, date_of_birth, address, state, phone_number, email, nationality, resident_status, marital_status, role, salary, hire_date, epf_number, socso_number, bank_account)
            VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
        ''', 
            (
                employee['id'],
                employee['age'],
                employee['name'],
                employee['gender'],
                employee['date_of_birth'],
                employee['address'],
                employee['state'],
                employee['phone_number'],
                employee['email'],
                employee['nationality'],
                employee['resident_status'],
                employee['marital_status'],
                employee['role'],
                employee['salary'],
                employee['hire_date'],
                employee['epf_number'],
                employee['socso_number'],
                employee['bank_account']
            )
    )
                
conn.commit()
conn.close()
                 

In [7]:
# delete the table employees
#conn = sqlite3.connect('database/database.db')
#c = conn.cursor()
#c.execute('DROP TABLE employees')
#c.commit()